In [5]:
!ls workshop/safe-smart-contracts-workshop-08-2015/example-04-micropayments.py

workshop/safe-smart-contracts-workshop-08-2015/example-04-micropayments.py


In [6]:
!cat workshop/safe-smart-contracts-workshop-08-2015/example-04-micropayments.py

from ethereum import tester, utils
import bitcoin
import os

contract_code = """
event Notice(x:str)

alice = 0x{alice}
bob = 0x{bob}
deadline = {deadline}

macro verify_signature($addr, $m, $v, $r, $s):
    $addr == ecrecover(sha3($m), $v, $r, $s)

def refund():
   if msg.sender != alice:
      log(type=Notice, text("refund called by other-than-Alice"))
      return(-1)

   if block.number < deadline: 
      log(type=Notice, text("Too soon for Alice to claim refund"))
      return(-1)

   send(alice, self.balance)

def finalize(v,r,s, amount):
   if msg.sender != bob:
      log(type=Notice, text("finalize called by other-than-Bob"))
      return(-1)

   if !verify_signature(alice, amount, v, r, s):
      log(type=Notice, text("bad signature!"))
      return(-1)

   send(bob, amount)
   send(alice, self.balance)

"""

s = tester.state()

# Use default addresses for Alice and Bob
alice = tester.a0
bob = tester.a1

print 'Initial balances:'
print 'Alice: %.2f' % (float(s.block.get_balanc